In [63]:
import json
import pandas as pd
import scipy as sp

study_id = "MGYS00005628"

data = pd.read_table(f"data/mgnify_data/{study_id}__ERP005534_GO_abundances_v5.0.tsv", sep="\t", header=0, index_col=[0,1,2])
data = data.droplevel(["GO", "category"])
data = data.T
display(data.head())

with open(f"data/mgnify_data/{study_id}_metadata.json") as f:
    metadata = json.load(f)

metadata = pd.DataFrame(metadata["metadata"])
metadata = metadata.loc["Health state__elixir"]
metadata = metadata.to_frame().T
display(metadata.head())
print(metadata.T.value_counts(dropna=False))
print(len(metadata.T))



description,mitochondrion inheritance,mitochondrial genome maintenance,vacuole inheritance,single strand break repair,phosphopyruvate hydratase complex,mannosyltransferase activity,transition metal ion transport,autophagosome assembly,tRNA binding,fatty-acyl-CoA binding,...,Lys48-specific deubiquitinase activity,queuosine salvage,acetolactate synthase regulator activity,CST complex,ribonucleoprotein complex,drug transmembrane export,positive regulation of stomatal complex development,negative regulation of microtubule motor activity,starch binding,regulation of store-operated calcium entry
ERR478960,0,0,0,0,6,0,1,0,48,0,...,0,0,2,0,0,0,0,0,2,0
ERR478970,0,0,0,0,8,2,4,0,70,0,...,0,11,3,0,0,0,0,0,7,0
ERR478980,0,0,0,0,1,0,0,0,12,0,...,0,1,1,0,0,0,0,0,0,0
ERR478990,0,0,0,0,4,0,1,0,52,0,...,0,5,5,0,1,0,0,0,2,0
ERR478961,0,0,0,0,4,0,1,0,61,0,...,0,2,1,0,0,0,0,0,2,0


,ERS436836,ERS436735,ERS436823,ERS436725,ERS436833,ERS436767,ERS436831,ERS436763,ERS436733,ERS436681,...,ERS581038,ERS581044,ERS581047,ERS436757,ERS433421,ERS433485,ERS433490,ERS581054,ERS433501,ERS581033
Health state__elixir,Diseased,Healthy,Diseased,Diseased,Diseased,Healthy,Diseased,Diseased,Diseased,Diseased,...,NaN,NaN,NaN,Diseased,NaN,NaN,NaN,NaN,NaN,NaN


Health state__elixir
NaN                     225
Diseased                136
Healthy                  63
Name: count, dtype: int64
424


combined the datasets and filter to keep only the ones with the useful label

In [64]:
%load_ext autoreload
%autoreload 2
import mgnify_helper_functions as mhf

mgnify = mhf.MGnifyData(cache_folder="data/mgnify_data")
assembly_id_to_sample_id = mgnify.get_run_id_to_sample_id_dict(study_id)

column_mapper = {}
for key, value in assembly_id_to_sample_id.items():
    if value not in column_mapper:
        column_mapper[value] = []
    column_mapper[value].append(key)

new_columns = {new_name: metadata[old_name] for old_name, new_names in column_mapper.items() for new_name in new_names}
metadata = metadata.drop(columns=list(column_mapper.keys()))
metadata = metadata.join(pd.DataFrame(new_columns))
metadata = metadata.dropna(axis=1, how="all")
metadata = metadata.T
display(metadata)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,Health state__elixir
ERS436810,Diseased
ERS436709,Diseased
ERS436644,Healthy
ERS436741,Diseased
ERS436720,Diseased
...,...
ERR479505,Diseased
ERR479503,Diseased
ERR479502,Diseased
ERR479501,Diseased


In [66]:
print("data points: ", len(data))
print("metadata points: ", len(metadata))

print(metadata.value_counts())

combined_data = data.join(metadata, how="inner") # outer or left or inner? Which one do we want

print("combined data points: ", len(combined_data))
display(combined_data.head())

data points:  644
metadata points:  535
Health state__elixir
Diseased                337
Healthy                 198
Name: count, dtype: int64
combined data points:  525


,mitochondrion inheritance,mitochondrial genome maintenance,vacuole inheritance,single strand break repair,phosphopyruvate hydratase complex,mannosyltransferase activity,transition metal ion transport,autophagosome assembly,tRNA binding,fatty-acyl-CoA binding,...,queuosine salvage,acetolactate synthase regulator activity,CST complex,ribonucleoprotein complex,drug transmembrane export,positive regulation of stomatal complex development,negative regulation of microtubule motor activity,starch binding,regulation of store-operated calcium entry,Health state__elixir
ERR478970,0,0,0,0,8,2,4,0,70,0,...,11,3,0,0,0,0,0,7,0,Healthy
ERR478980,0,0,0,0,1,0,0,0,12,0,...,1,1,0,0,0,0,0,0,0,Healthy
ERR478990,0,0,0,0,4,0,1,0,52,0,...,5,5,0,1,0,0,0,2,0,Healthy
ERR478961,0,0,0,0,4,0,1,0,61,0,...,2,1,0,0,0,0,0,2,0,Healthy
ERR478971,0,0,0,0,6,1,3,0,72,1,...,6,1,0,0,0,0,0,3,0,Healthy


<font color="red">We need to be careful, as many samples have multiple runs. value_count on metadata only, shows in total 136+63 samples with Health state available. It is much more here.</font>

##### Outlier detection

In [ ]:
from scipy import stats
import numpy as np

data.describe()

outliers = (np.abs(stats.zscore(data)) < 3).all(axis=1)
print(outliers.sum())

221


<font color="red">I guess this is kind of expected? A lot of zeros for some features namely</font>

### Split data

In [ ]:
from sklearn.model_selection import train_test_split

X, y = combined_data.drop(columns="Health state__elixir"), combined_data["Health state__elixir"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### batch-effect correction

### Imputation

### Normalize

In [22]:
from sklearn.preprocessing import normalize

data = pd.DataFrame(normalize(data, norm="l1"), index=data.index, columns=data.columns)
display(data.describe())


description,mitochondrion inheritance,mitochondrial genome maintenance,vacuole inheritance,single strand break repair,phosphopyruvate hydratase complex,mannosyltransferase activity,transition metal ion transport,autophagosome assembly,tRNA binding,fatty-acyl-CoA binding,...,Lys48-specific deubiquitinase activity,queuosine salvage,acetolactate synthase regulator activity,CST complex,ribonucleoprotein complex,drug transmembrane export,positive regulation of stomatal complex development,negative regulation of microtubule motor activity,starch binding,regulation of store-operated calcium entry
count,6.440000e+02,6.440000e+02,6.440000e+02,6.440000e+02,644.000000,644.000000,644.000000,6.440000e+02,644.000000,644.000000,...,6.440000e+02,644.000000,644.000000,6.440000e+02,6.440000e+02,6.440000e+02,6.440000e+02,6.440000e+02,644.000000,6.440000e+02
mean,3.657050e-07,1.674850e-07,1.891737e-07,6.045056e-08,0.000088,0.000012,0.000046,3.119967e-08,0.001320,0.000004,...,4.334173e-08,0.000097,0.000036,2.976105e-08,1.260184e-07,8.163352e-07,1.532297e-08,2.040209e-08,0.000064,3.432706e-09
std,3.304958e-06,1.556991e-06,2.681891e-06,1.308198e-06,0.000052,0.000028,0.000045,4.686103e-07,0.000270,0.000012,...,4.806931e-07,0.000058,0.000034,4.439606e-07,1.330784e-06,4.512071e-06,2.749709e-07,2.192477e-07,0.000050,8.711232e-08
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000425,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000064,0.000000,0.000019,0.000000e+00,0.001180,0.000000,...,0.000000e+00,0.000061,0.000014,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000030,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000084,0.000004,0.000038,0.000000e+00,0.001312,0.000000,...,0.000000e+00,0.000093,0.000030,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000054,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000108,0.000013,0.000064,0.000000e+00,0.001432,0.000001,...,0.000000e+00,0.000126,0.000049,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000089,0.000000e+00
max,6.384473e-05,2.040289e-05,6.531679e-05,3.300984e-05,0.000428,0.000374,0.000428,8.121867e-06,0.004587,0.000098,...,9.709021e-06,0.000428,0.000286,1.040420e-05,2.376652e-05,8.218277e-05,5.132943e-06,3.248747e-06,0.000321,2.210663e-06
